# Preprocessing WildType Sequences with Point Mutations

## Sample

Load the varient scores and codon counts

In [3]:
import pandas as pd
from Bio import SeqIO
import os

current_dir =  os.getcwd()
data_dir = os.path.join(current_dir, '..','data', 'processed','results')
variant_scores_path = os.path.join(data_dir, 'final_variant_scores', 'final_variant_scores.csv')
codon_counts_path = os.path.join(data_dir, 'variants','codon_variant_table_Wuhan_Hu_1.csv')

df_variant_scores = pd.read_csv(variant_scores_path)

df_codon_counts = pd.read_csv(codon_counts_path)

df_codon_counts.head()

,target,library,barcode,variant_call_support,codon_substitutions,aa_substitutions,n_codon_substitutions,n_aa_substitutions
0,Wuhan_Hu_1,pool1,AAAAAAAAAAAGGAGA,4,GGT166ATG,G166M,1,1
1,Wuhan_Hu_1,pool1,AAAAAAAAAAATTTAA,4,NaN,NaN,0,0
2,Wuhan_Hu_1,pool1,AAAAAAAAAACGCGTA,3,GAA154ACT,E154T,1,1
3,Wuhan_Hu_1,pool1,AAAAAAAAAACTCCAA,2,TTT156ATG,F156M,1,1
4,Wuhan_Hu_1,pool1,AAAAAAAAACCGATTA,2,CAG84GAA,Q84E,1,1


Our column of interest is ```aa_substitutions```. The way the substituitions are encoded is original amino acid | position in sequence | mutated amino acid. To improve overall context for the transformer model, we will subsitute those mutations into the entire amino acid wildtype sequence. Additionally, multiple mutations in one sequence are encoded as a string with a space between each mutation, i.e: ```'C6F S29H S36D'``` like below. Our next column of interest below is ```'bind'``` which is the binding affinity of that sequence.

In [4]:
df_variant_scores.head()

,target,wildtype,position,mutant,mutation,bind,delta_bind,n_bc_bind,n_libs_bind,bind_rep1,bind_rep2,bind_rep3,expr,delta_expr,n_bc_expr,n_libs_expr,expr_rep1,expr_rep2
0,Beta,N,331,A,N331A,9.41007,0.10297,6,3,9.31807,9.49961,9.41253,9.72294,-0.27844,3,2,9.63034,9.81554
1,Beta,N,331,C,N331C,9.11229,-0.19481,27,3,9.06925,9.09584,9.17180,9.36532,-0.63606,17,2,9.18919,9.54145
2,Beta,N,331,D,N331D,9.31717,0.01007,23,3,9.18921,9.38256,9.37976,9.77842,-0.22296,15,2,9.70504,9.85180
3,Beta,N,331,E,N331E,9.36899,0.06189,17,3,9.36041,9.42835,9.31821,10.06567,0.06430,10,2,9.93181,10.19954
4,Beta,N,331,F,N331F,9.17224,-0.13487,22,3,9.07594,9.17972,9.26104,9.42578,-0.57560,14,2,9.30185,9.54970


The wildtype sequence is stored in ```'wildtype_sequence.fasta'``` in the ```data/raw/``` directory.

In [6]:
#load wildtype codon sequence
wt_sequence_path = os.path.join(current_dir,'..', 'data', 'raw','wildtype_sequence.fasta')

#skip the first line
wt_nuc_seq = next(SeqIO.parse(wt_sequence_path, "fasta")).seq

#translate codon sequence to amino acid sequence
wt_aa_seq = wt_nuc_seq.translate(to_stop=True)

#visualize
wt_aa_seq

Seq('NITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSP...KST')

Lets perform a sequence substitution with the example from above.

In [7]:
#original wt Amino acid sequence
seq = list(str(wt_aa_seq))

#sample mutations in one sequence
mutations = ['C6F', 'S29H', 'S36D']

for mut in mutations:
    from_aa = mut[0]
    to_aa = mut[-1]
    pos = int(mut[1:-1]) -1 # the mutations are indexed starting at 1 not 0
    assert seq[pos] == from_aa, f"expected {from_aa} at position {pos+1}, got {seq[pos]}"
    seq[pos] = to_aa

print(''.join(seq))




NITNLFPFGEVFNATRFASVYAWNRKRIHNCVADYDVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKST


If you look closely at the output, you can see that ```C``` at position 6 has been swapped with ```F```, etc. These will be the sequences we use for modeling. See ```src/utils``` and ```scripts``` for preprocessing details.

In [8]:
bc_binding_file = os.path.join(data_dir, 'binding_Kd', 'bc_binding.csv')

bc_binding = pd.read_csv(bc_binding_file)

bc_binding.head()


,library,barcode,target,variant_class,aa_substitutions,n_aa_substitutions,TiteSeq_avgcount,log10Ka
0,pool1A,AAAAAAAAAAAAAGAA,N501Y,1 nonsynonymous,T46Q,1,1.955927,NaN
1,pool1A,AAAAAAAAAAAGGAGA,Wuhan_Hu_1,1 nonsynonymous,G166M,1,0.000000,NaN
2,pool1A,AAAAAAAAAAATTTAA,Wuhan_Hu_1,wildtype,NaN,0,36.004828,8.506648
3,pool1A,AAAAAAAAAACGCGTA,Wuhan_Hu_1,1 nonsynonymous,E154T,1,26.968699,8.586023
4,pool1A,AAAAAAAAAACTCCAA,Wuhan_Hu_1,1 nonsynonymous,F156M,1,40.906723,7.698023


In [10]:
bc_binding['aa_substitutions'].value_counts(dropna=False)

aa_substitutions
NaN           134261
V115L            382
V32L             328
V115S            316
V115R            296
               ...  
S29Y A81D          1
T15A N120R         1
N40W Q144W         1
A14R L111C         1
P54D L183H         1
Name: count, Length: 47254, dtype: int64